In [1]:
import sys
import numpy as np
import scipy
py_path = '/home/haorui/moco-test'
if py_path not in sys.path:
    sys.path.append(py_path)
# from graphsaint.mocodict.encoder_predict import encoder_predict
%run /home/haorui/moco-test/graphsaint/mocodict/encoder_predict.py
import pandas as pd
import scipy as sc
import networkx as nx
import numpy as np
import os
from IPython.display import display
from multiprocessing import  Pool
from tqdm import tqdm
from time import time
from torch.utils.data import Dataset, DataLoader


# Load stock data

In [2]:
df = pd.read_pickle('22mdata.pkl')
df

,kdcode,dt,open,close,high,low,turnover,volume
0,000001.SH,2020-01-02,3066.3357,3085.1976,3098.1001,3066.3357,3.271971e+11,2.924702e+10
1,000001.SH,2020-01-03,3089.0220,3083.7858,3093.8192,3074.5178,2.899917e+11,2.614967e+10
2,000001.SH,2020-01-06,3070.9088,3083.4083,3107.2032,3065.3088,3.311825e+11,3.125758e+10
3,000001.SH,2020-01-07,3085.4882,3104.8015,3105.4507,3084.3290,2.881592e+11,2.765831e+10
4,000001.SH,2020-01-08,3094.2389,3066.8925,3094.2389,3059.1313,3.065174e+11,2.978726e+10
...,...,...,...,...,...,...,...,...
1974404,SPX.GI,2020-02-04,3280.6100,3297.5900,3306.9200,3280.6100,NaN,3.995320e+09
1974405,SPX.GI,2020-02-05,3324.9100,3334.6900,3337.5800,3313.7500,NaN,4.117730e+09
1974406,SPX.GI,2020-02-06,3344.9200,3345.7800,3347.9600,3334.3900,NaN,3.868370e+09
1974407,SPX.GI,2020-02-07,3335.5400,3327.7100,3341.4200,3322.1200,NaN,3.730650e+09


## Add label

In [3]:
df_open2 = df['open'].iloc[1:].values
df_open1 = df['close'].iloc[:-1].values

pre_label = df_open2 - df_open1
pre_label[pre_label >= 0] = 1
pre_label[pre_label < 0] = 0
pre_label = list(pre_label)
pre_label.append(0)
df['label'] = pre_label
df

,kdcode,dt,open,close,high,low,turnover,volume,label
0,000001.SH,2020-01-02,3066.3357,3085.1976,3098.1001,3066.3357,3.271971e+11,2.924702e+10,1.0
1,000001.SH,2020-01-03,3089.0220,3083.7858,3093.8192,3074.5178,2.899917e+11,2.614967e+10,0.0
2,000001.SH,2020-01-06,3070.9088,3083.4083,3107.2032,3065.3088,3.311825e+11,3.125758e+10,1.0
3,000001.SH,2020-01-07,3085.4882,3104.8015,3105.4507,3084.3290,2.881592e+11,2.765831e+10,0.0
4,000001.SH,2020-01-08,3094.2389,3066.8925,3094.2389,3059.1313,3.065174e+11,2.978726e+10,1.0
...,...,...,...,...,...,...,...,...,...
1974404,SPX.GI,2020-02-04,3280.6100,3297.5900,3306.9200,3280.6100,NaN,3.995320e+09,1.0
1974405,SPX.GI,2020-02-05,3324.9100,3334.6900,3337.5800,3313.7500,NaN,4.117730e+09,1.0
1974406,SPX.GI,2020-02-06,3344.9200,3345.7800,3347.9600,3334.3900,NaN,3.868370e+09,0.0
1974407,SPX.GI,2020-02-07,3335.5400,3327.7100,3341.4200,3322.1200,NaN,3.730650e+09,0.0


## Normalization

In [4]:
nor_colomns = ['open', 'close', 'high', 'low', 'turnover', 'volume']
res_df = df[nor_colomns]
tmp_df = res_df.copy()

In [5]:
from sklearn.preprocessing import RobustScaler

robustScaler = RobustScaler()
df_nor = robustScaler.fit_transform(res_df)
df_nor = pd.DataFrame(df_nor, columns=res_df.columns)

df_sub_columns = [c for c in df.columns if c not in df_nor.columns]
df_nor[df_sub_columns] = df[df_sub_columns]
df_nor = df_nor[df.columns]
df_nor = df_nor.fillna(0)
df_nor

,kdcode,dt,open,close,high,low,turnover,volume,label
0,000001.SH,2020-01-02,145.638358,146.359979,143.641605,149.039302,1321.877388,1412.140379,1.0
1,000001.SH,2020-01-03,146.719828,146.292759,143.442386,149.438429,1171.526857,1262.550382,0.0
2,000001.SH,2020-01-06,145.856360,146.274785,144.065234,148.989210,1337.982870,1509.242669,1.0
3,000001.SH,2020-01-07,146.551369,147.293380,143.983678,149.917024,1164.121633,1335.411961,0.0
4,000001.SH,2020-01-08,146.968521,145.488418,143.461917,148.687868,1238.308688,1438.231577,1.0
...,...,...,...,...,...,...,...,...,...
1974404,SPX.GI,2020-02-04,155.852946,156.472626,153.359410,159.491707,0.000000,192.582736,1.0
1974405,SPX.GI,2020-02-05,157.964755,158.239070,154.786227,161.108293,0.000000,198.494656,1.0
1974406,SPX.GI,2020-02-06,158.918644,158.767098,155.269279,162.115122,0.000000,186.451551,0.0
1974407,SPX.GI,2020-02-07,158.471493,157.906731,154.964928,161.516585,0.000000,179.800218,0.0


## Index

In [6]:
df_nor['idx'] = df_nor['kdcode'] + ',' + df_nor['dt'].apply(str)
df_nor.set_index('idx', inplace=True)
df_nor['emb'] = None
df_nor['emb'] = df_nor['emb'].astype(object)

# G-MoCo

In [7]:
config_path = '/home/haorui/Downloads/fetch-data/log_train/22mnews/2021-11-25 00-44-51-pretrain/kd-news.yml'
model_path = '/home/haorui/Downloads/fetch-data/log_train/22mnews/2021-11-25 00-44-51-pretrain/pretrained_models/saved_encoder_2021-11-25 00-44-51.pkl'

In [8]:
def timer(func):
    # This function shows the execution time of 
    # the function object passed
    def wrap_func(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func

def parallelize_dataframe(df, func, n_cores=4):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def rel_orgs_wrap(text):
    return text.split('|') if text else []

def doc_to_feat(doc):
    return np.random.rand(64)



In [20]:
import time
class Timer:
    def __init__(self, on=True):
        self.on = on
        self.reset()

    def reset(self):
        self.tstart = time.time()

    def __repr__(self):
        t = time.time() - self.tstart
        return '%.1fs (%s)' % (t, time.strftime('%H:%M:%S', time.gmtime(t)))

    def __call__(self, name=None):
        if not self.on:
            return
        if name:
            print('[%s]' % name, end=' ')
        print('Elapsed: %s' % (self))
        self.tstart = time.time()

In [10]:
class NewsDataSet:
  def __init__(self, row_path, feat_path, data_path_list):
    self.row_path = row_path
    self.feat_path = feat_path
    self.data_path_list = data_path_list
    self.index = 0
    #if len(self.X) != len(self.Y):
    #  raise Exception("The length of X does not match the length of Y")
  def __len__(self):
    return len(self.data_path_list)
  
  def __iter__(self):
    return self

  def __next__(self):
    if self.index >= len(self.data_path_list):
      raise StopIteration
    index = self.index
    self.index += 1
    row_path = os.path.join(self.row_path, self.data_path_list[index]+'.pkl')
    feat_path = os.path.join(self.feat_path, self.data_path_list[index]+'.npy')
    df = pd.read_pickle(row_path)
    feat = np.load(feat_path, allow_pickle=True)
    _df = pd.DataFrame()
    _df['id'] = df['id']
    _df['rel_orgs'] = df['rel_org_a_companies_code'].map(rel_orgs_wrap)
    _df['date'] = df['date_time'].apply(lambda x: x.date())
    _df['feat'] = _df.apply(lambda x: feat[x.name], axis=1)
    df = _df
    return df


In [18]:
class TheGraph(nx.Graph):
    def doc_nodes(self):
        return [n for n in self.nodes if not isinstance(n, str)]
    def stock_nodes(self):
        return [n for n in self.nodes if isinstance(n, str)]
    def get_feats(self):
        return np.array([self.nodes[k]['feat'] for k in self.nodes])
    def add_doc_node(self, node_id, feat, adj_to=[], days=7):
        self.add_node(node_id, feat=feat, valid_days=days)
        for stock in adj_to:
            if stock not in self.nodes:
                self.add_node(stock, feat=np.zeros(feat.shape[0]))
            self.add_edge(node_id, stock)
    def get_adj_matrix(self):
        return nx.convert_matrix.to_scipy_sparse_matrix(self)
    def new_day(self):
        for node in self.doc_nodes():
            self.nodes[node]['valid_days'] -= 1
    def remove_outdated_nodes(self):
        for node in self.doc_nodes():
            if self.nodes[node]['valid_days'] <= 0:
                self.remove_node(node)
    def remove_isolated_nodes(self):
        self.remove_nodes_from(list(nx.isolates(self)))
    def clean_nodes(self):
        self.remove_outdated_nodes()
        self.remove_isolated_nodes()

    

In [24]:
data_sets = ['2020-1', '2020-2', '2020-3', '2020-4', '2020-5', '2020-6', '2020-7', '2020-8', '2020-9', '2020-10', '2020-11', '2020-12', '2021-1', '2021-2', '2021-3', '2021-4', '2021-5', '2021-6', '2021-7', '2021-8', '2021-9', '2021-10']
loader = NewsDataSet('data/news_row', 'data/news_embed',  data_sets)
G = TheGraph()

timer = Timer(False)
for df_month in loader:
    timer.reset()
    doc_df_grouped = {k:v for k, v in df_month.groupby('date')}
    timer('grouped')
    for date, docs in tqdm(doc_df_grouped.items()):
    # for date, docs in doc_df_grouped.items():
        G.new_day()
        timer.reset()
        for _, doc in docs.iterrows():
            G.add_doc_node(doc.id, doc.feat, doc.rel_orgs, days=14)
        timer('edges added')
        G.clean_nodes()
        feats = G.get_feats()
        timer('feats computed')
        adj = G.get_adj_matrix()
        timer('adj computed')
        output = encoder_predict(adj, feats, model_path, config_path).detach()
        timer('encoded')
        date_ = str(date)
        for kdnode, emb in zip(G.nodes, output):
            if isinstance(kdnode, str):
                k = kdnode+','+date_
                if k in df_nor.index:
                    df_nor.at[kdnode+','+date_, 'emb'] = emb.numpy()
        timer('emb added')
        timer(f'month done. nodes:{G.number_of_nodes()} edges:{G.number_of_edges()}')
        timer('------------------------')

100%|██████████| 31/31 [02:38<00:00,  5.12s/it]


In [ ]:
# df_nor.fillna(method='pad')
df_nor_grouped = [ x for _, x in df_nor.groupby('kdcode') ]
for s in df_nor_grouped:
    s['emb'] = s['emb'].fillna(method='pad', limit=7)
df_nor = pd.concat(df_nor_grouped)
del df_nor_grouped

In [25]:
df_nor.reset_index(drop=True)
_l = list(df_nor.columns)
_l.remove('label')
_l.append('label')
df_nor = df_nor[_l]

In [26]:
df_nor.to_pickle('22mnewsmoco.pkl')